# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [71]:
# import libraries
import pandas as pd
import numpy as np
import re
from time import time
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
import pickle
nltk.download(['punkt', 'wordnet', 'stopwords'])
from collections import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kroening\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kroening\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kroening\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
table_name = "messages"
df = pd.read_sql_table(table_name, engine)
X = df.message
Y = df.drop(['id','message','genre','original'], axis=1)

In [3]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [35]:
Y.head()

related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone      ...        \
0                  0         0         0            0      ...         
1                  0         0         0            0      ...         
2                  0         0         0            0      ...         
3                  0         0         0            0      ...         
4                  0         0         0            0      ...         

   aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
0            0                     0                0       0      0     0   
1            0                     0                1       0      1     0   
2            0                     0                0       0      0     0   
3            0                     0                0       0      0     0   
4            0                     0                0       0      0     0   

   earthquake  cold  other_weather  direct_report  
0           0     0              0              0  
1           0     0              0              0  
2           0     0              0              0  
3           0     0              0              0  
4           0     0              0              0  

[5 rows x 36 columns]

### 2. Write a tokenization function to process your text data

In [23]:
def tokenize(text):
    """
    Function: tokenize text
    Args:
        text (str): string to tokenize
    Return:
        clean_tokens (list of str): list of cleaned and tokenized strings
    """
    # 1. normalization
    # remove punctuation and capitalization of words
    text = re.sub(r'[^a-zA-Z0-9]', " ", text.lower())
    
    # 2. tokenization
    # tokenize words from text
    tokens = word_tokenize(text)
    
    # 3. stop word removal
    # remove all words that can be found in the stopwords dictionary
    tokens = [w for w in tokens if w not in stopwords.words("english")]

    # 4. stemming and lemmtization
    # define lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        # lemmatize and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [37]:
# check funtion
print("Message: ", X[0])
print("Tokens: ", tokenize(X[0]))

Message:  Weather update - a cold front from Cuba that could pass over Haiti
Tokens:  ['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [38]:
# define ML pipeline with TF-IDF Vectorizer and MultiOutputClassifier (as estimator we use a RandomForestClassifier)
pipeline1 = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [39]:
# split dataset into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.20, random_state=42)

# train pipeline
start = time()
pipeline1.fit(X_train, y_train)
print("Training Time: {:.0f} seconds".format(time()-start))

Training Time: 447 seconds


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [40]:
def display_accuracy(y_test, y_pred):
    """
    Function: show accuracy metrics for predicted test dataset (f1_score, precision, recall, support)
    Args:
        y_test (pd.Series): original labels of test dataset
        y_pred (pd.Series): predicted labels of test dataset
    Return:
        None
    """
    for i, col in enumerate(y_test):
        print(col,':')
        print(classification_report(y_test[col], y_pred[:, i]))

In [41]:
y_pred1 = pipeline1.predict(X_test)

In [42]:
display_accuracy(y_test, y_pred1)

related :
              precision    recall  f1-score   support

           0       0.70      0.41      0.52      1245
           1       0.84      0.94      0.89      3998

   micro avg       0.82      0.82      0.82      5243
   macro avg       0.77      0.68      0.70      5243
weighted avg       0.80      0.82      0.80      5243

request :
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4352
           1       0.85      0.49      0.62       891

   micro avg       0.90      0.90      0.90      5243
   macro avg       0.87      0.73      0.78      5243
weighted avg       0.89      0.90      0.89      5243

offer :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      1.00      0.99   

/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [43]:
# print overall accuracy, precision and recall of tuned model
print('accuracy:\t{:.4}'.format(accuracy_score(y_test,y_pred)))
print('precision:\t{:.4}'.format(precision_score(y_test,y_pred, average='weighted')))
print('recall:\t\t{:.4}'.format(recall_score(y_test,y_pred, average='weighted')))

accuracy:	0.2785
precision:	0.7654
recall:		0.5345


/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [44]:
# inspect pipeline parameters
pipeline1.get_params(deep=True)

{'memory': None,
 'steps': [('vect',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd1bd02b048>, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
               oob_score=False, random_state=None, verbo

In [45]:
# define selection of parameters for GridSearch
# we limit ourselves to only three parameters since the training time will massively increase

parameters = {
    'vect__ngram_range': [(1,1),(1,2)],
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_split': [2, 4]
}

cv1 = GridSearchCV(pipeline, param_grid=parameters, cv=3, n_jobs=1)

In [46]:
# train GridSearch object
start = time()
cv1.fit(X_train, y_train)
print("Training Time: {:.0f} seconds".format(time()-start))

Training Time: 9474 seconds


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [70]:
# predict output of test dataset with tuned model
y_pred_cv1 = cv1.best_estimator_.predict(X_test)

In [78]:
print(classification_report(y_test, y_pred_cv1, target_names=list(y_test.columns)))

                        precision    recall  f1-score   support

               related       0.85      0.94      0.89      3998
               request       0.83      0.47      0.60       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.79      0.63      0.70      2164
          medical_help       0.66      0.06      0.11       435
      medical_products       0.84      0.10      0.17       279
     search_and_rescue       0.73      0.08      0.15       136
              security       0.33      0.01      0.02        96
              military       0.56      0.03      0.06       158
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.49      0.62       335
                  food       0.86      0.55      0.67       584
               shelter       0.83      0.32      0.47       468
              clothing       0.89      0.11      0.20        70
                 money       0.88      

/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [49]:
# print overall accuracy, precision and recall of tuned model
print('accuracy:\t{:.4}'.format(accuracy_score(y_test,y_pred_cv1)))
print('precision:\t{:.4}'.format(precision_score(y_test,y_pred_cv1, average='weighted')))
print('recall:\t\t{:.4}'.format(recall_score(y_test,y_pred_cv1, average='weighted')))

accuracy:	0.2884
precision:	0.7606
recall:		0.5028


/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [81]:
# print parameters of tuned model
cv1.best_estimator_.get_params()

{'memory': None,
 'steps': [('vect',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd1bc8c1bf8>, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
               oob_score=False, random_state=None, verbo

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [51]:
# define new pipeline with another ML algorithm
pipeline2 = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))    
])

In [56]:
# train pipeline
start = time()
pipeline2.fit(X_train, y_train)
print("Training Time: {:.0f} seconds".format(time()-start))

Training Time: 137 seconds


In [57]:
y_pred2 = pipeline2.predict(X_test)

In [58]:
display_accuracy(y_test, y_pred2)

related :
              precision    recall  f1-score   support

           0       0.70      0.24      0.36      1245
           1       0.80      0.97      0.88      3998

   micro avg       0.80      0.80      0.80      5243
   macro avg       0.75      0.61      0.62      5243
weighted avg       0.78      0.80      0.76      5243

request :
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      4352
           1       0.78      0.48      0.59       891

   micro avg       0.89      0.89      0.89      5243
   macro avg       0.84      0.72      0.76      5243
weighted avg       0.88      0.89      0.88      5243

offer :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5219
           1       0.00      0.00      0.00        24

   micro avg       0.99      0.99      0.99      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      0.99      0.99   

In [59]:
# print overall accuracy, precision and recall of tuned model
print('accuracy:\t{:.4}'.format(accuracy_score(y_test,y_pred2)))
print('precision:\t{:.4}'.format(precision_score(y_test,y_pred2, average='weighted')))
print('recall:\t\t{:.4}'.format(recall_score(y_test,y_pred2, average='weighted')))

accuracy:	0.2565
precision:	0.7347
recall:		0.5789


/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [60]:
# inspect pipeline parameters
pipeline2.get_params(deep=True)

{'memory': None,
 'steps': [('vect',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd1bd02b048>, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=None))],
 'vect': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1,

In [61]:
# define selection of parameters for GridSearch
parameters = {
    'vect__max_df': [0.9, 1.0],
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__learning_rate':[0.9, 1.0]
}

cv2 = GridSearchCV(pipeline2, param_grid=parameters, cv=3, n_jobs=1)

In [62]:
# train new GridSearch object
start = time()
cv2.fit(X_train, y_train)
print("Training Time: {:.0f} seconds".format(time()-start))

Training Time: 4498 seconds


In [68]:
# predict output of test dataset with new tuned model
y_pred_cv2 = cv2.best_estimator_.predict(X_test)

In [79]:
print(classification_report(y_test, y_pred_cv2, target_names=list(y_test.columns)))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      3998
               request       0.78      0.53      0.63       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.78      0.62      0.69      2164
          medical_help       0.63      0.27      0.38       435
      medical_products       0.64      0.29      0.40       279
     search_and_rescue       0.53      0.14      0.22       136
              security       0.08      0.01      0.02        96
              military       0.57      0.32      0.41       158
           child_alone       0.00      0.00      0.00         0
                 water       0.73      0.66      0.69       335
                  food       0.82      0.69      0.75       584
               shelter       0.74      0.56      0.63       468
              clothing       0.74      0.36      0.48        70
                 money       0.49      

/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [69]:
# print overall accuracy, precision and recall of tuned model
print('accuracy:\t{:.4}'.format(accuracy_score(y_test,y_pred_cv2)))
print('precision:\t{:.4}'.format(precision_score(y_test,y_pred_cv2, average='weighted')))
print('recall:\t\t{:.4}'.format(recall_score(y_test,y_pred_cv2, average='weighted')))

accuracy:	0.2644
precision:	0.7354
recall:		0.586


/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [82]:
# print parameters of tuned model
cv2.best_estimator_.get_params()

{'memory': None,
 'steps': [('vect',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd1bd02b048>, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=0.9, n_estimators=100, random_state=None),
              n_jobs=None))],
 'vect': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1

### 9. Export your model as a pickle file

In [84]:
# since the f1_score of the second model tuned with GridSearch is better than the first one,
# we will export cv2
with open('../models/disaster_response_clf.pkl', 'wb') as f:
    pickle.dump(cv2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [88]:
# import libraries
import pandas as pd
import numpy as np
import re
from time import time
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
nltk.download(['punkt', 'wordnet', 'stopwords'])

def load_data(db_path = '../data/DisasterResponse.db', table_name = "messages"):
    '''
    Function:   load data from SQLite database and return X and y variables
    Args:
        data_file (str):    path of SQLite database
        table_name (str):   name of table in SQLite database
    Return:
        X (pd.DataFrame):   messages (features)
        y (pd.DataFrame):   categories (labels)
    '''
    engine = create_engine('sqlite:///'+db_path)
    df = pd.read_sql_table(table_name, engine)
    X = df.message
    y = df.drop(['id','message','genre','original'], axis=1)
    return X, y

def tokenize(text):
    """
    Function: tokenize text
    Args:
        text (str): string to tokenize
    Return:
        clean_tokens (list of str): list of cleaned and tokenized strings
    """
    # 1. normalization
    # remove punctuation and capitalization of words
    text = re.sub(r'[^a-zA-Z0-9]', " ", text.lower())
    
    # 2. tokenization
    # tokenize words from text
    tokens = word_tokenize(text)
    
    # 3. stop word removal
    # remove all words that can be found in the stopwords dictionary
    tokens = [w for w in tokens if w not in stopwords.words("english")]

    # 4. stemming and lemmtization
    # define lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        # lemmatize and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

def build_model():
    """
    Function: create GridSearch model consisting of TF-IDF-Vectorizer and AdaBoostClassifier
    Args:
        None
    Return:
        cv (GridSearch): GridSearchObject
    """
    # create pipeline
    pipeline = Pipeline([
        ('vect', TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    
    # define parameters for GridSearch
    parameters = {
        'vect__max_df': [0.9, 1.0],
        'clf__estimator__n_estimators': [50, 100],
        'clf__estimator__learning_rate':[0.9, 1.0]
    }
    
    # create model
    cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, n_jobs=1)
    
    return cv

def evaluate_model(model, X_test, y_test):
    """
    Function: predict output of model and evaluates the result against the testing dataset
    Args:
        model (classifier obj): trained classifier model
        X_test (pd.DataFrame): features of testing dataset
        y_test (pd.DataFrame): labels of testing dataset
    Return:
       None
    """
    # make predictions
    y_pred = model.predict(X_test)
    
    # create classification report
    print(classification_report(y_test, y_pred, target_names=list(y_test.columns)))
    
    # print total accuracy, precision, recall on testing dataset
    print('accuracy:\t{:.4}'.format(accuracy_score(y_test,y_pred)))
    print('precision:\t{:.4}'.format(precision_score(y_test,y_pred, average='weighted')))
    print('recall:\t\t{:.4}'.format(recall_score(y_test,y_pred, average='weighted')))
    
def export_model(model, clf_path = '../models/disaster_response_clf.pkl'):
    """
    Function: save model as pickle file
    Args:
        model (GridSearch obj): trained and tuned classifier model
        clf_path (str): path of pickle file destination
    Return:
        None
    """
    with open(clf_path, 'wb') as f:
        pickle.dump(model, f)

        
def import_model(clf_path = '../models/disaster_response_clf.pkl'):
    """
    Function: load model from pickle file
    Args:
        clf_path (str): path of pickle file
    Return:
        model (GridSearch obj): loaded model
    """
    with open(clf_path, 'rb') as f:
        model = pickle.load(f)
    
    return model

def main():
    print("Loading data...")
    X,y = load_data()
    print("Data loaded!")
    print("------------")
    
    print("Building model...")
    model = build_model()
    print("Model built!")
    print("------------")
    
    print("Training model...")
    start = time()
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state=42)
    model.fit(X_train, y_train)
    print("Training Time: {:.0f} seconds".format(time()-start))
    print("Model trained!")
    print("------------")
    
    print("Evaluating model...")
    evaluate_model(model, X_test, y_test)
    print("Model evaluated!")
    print("------------")
    
    print("Saving model...")
    clf_path = '../models/disaster_response_clf1.pkl'
    export_model(model.best_estimator_, clf_path)
    print("Model saved at ", clf_path, "!")
    print("------------")

if __name__== "__main__":
    main()
    
    

[nltk_data] Downloading package punkt to /home/oliver/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/oliver/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/oliver/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Loading data...
Data loaded!
------------
Building model...
Model built!
------------
Training model...
Training Time: 4575 seconds
Model trained!
------------
Evaluating model...


/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/oliver/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      3998
               request       0.78      0.53      0.63       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.78      0.62      0.69      2164
          medical_help       0.63      0.27      0.38       435
      medical_products       0.64      0.29      0.40       279
     search_and_rescue       0.53      0.14      0.22       136
              security       0.08      0.01      0.02        96
              military       0.57      0.32      0.41       158
           child_alone       0.00      0.00      0.00         0
                 water       0.73      0.66      0.69       335
                  food       0.82      0.69      0.75       584
               shelter       0.74      0.56      0.63       468
              clothing       0.74      0.36      0.48        70
                 money       0.49      

In [72]:
# TEST: function for counting words to create a visualization with the top 25 words within the web application

def count_words(df):
    '''
    Function: counts all tokens and returns the top 25 words
    Args:
        df (pd.DataFrame): DataFrame containing all messages
    Return:
        top_words (Counter): top 25 words and their frequency
    '''
    # create Counter object
    words = Counter()
    # iterate through messages in DataFrame
    for message in df.message:
        # tokenize each message
        tokens = tokenize(message)
        # iterate through created tokens
        for token in tokens:
            words[token] += 1
    # filter top 25 words
    top_words = words.most_common(25)
    return top_words 

In [73]:
# get most frequent 25 words
start = time()
top_words = count_words(df)
print("{:.0f} seconds".format(time()-start))
top_words

334 seconds


[('water', 3040),
 ('people', 3014),
 ('food', 2904),
 ('help', 2653),
 ('need', 2496),
 ('please', 2049),
 ('earthquake', 1921),
 ('u', 1757),
 ('area', 1667),
 ('like', 1532),
 ('would', 1491),
 ('said', 1351),
 ('http', 1257),
 ('000', 1254),
 ('country', 1250),
 ('know', 1116),
 ('also', 1114),
 ('government', 1056),
 ('haiti', 1043),
 ('one', 1020),
 ('rain', 1012),
 ('flood', 995),
 ('information', 974),
 ('year', 957),
 ('house', 929)]

In [70]:
# save most frequent 25 words and their frequency in text file
with open('../data/top_words.txt', 'w') as f:
    for item in top_words:
        f.write("{}\n".format(item))